# Q2: Top 10 emojis más usados

En este notebook puedes ejecutar y probar las funciones implementadas para la pregunta 2.

Aquí puedes medir el tiempo, memoria y validar los resultados.

In [ ]:
# Ruta al dataset
file_path = "../dataset/farmers-protest-tweets-2021-2-4.json"

In [ ]:
# Importar las funciones
from q2_time import q2_time
from q2_memory import q2_memory

In [ ]:
# Ejecutar y medir q2_time
result_time = q2_time(file_path)
print(result_time)

In [ ]:
# Ejecutar y medir q2_memory
result_memory = q2_memory(file_path)
print(result_memory)